In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from xgboost import XGBRegressor
from datetime import date, datetime
from sklearn.impute import SimpleImputer
from IPython.core.display import display, HTML
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
%matplotlib inline
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))
print('Libraries imported successfully!!')

Libraries imported successfully!!


In [2]:
db = pd.read_csv('iplav.csv')
db.head()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
0,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,0,0.1,1,0,0,0,222
1,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,0,0.2,1,0,0,0,222
2,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.2,2,0,0,0,222
3,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.3,2,0,0,0,222
4,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.4,2,0,0,0,222


In [3]:
db.drop(['mid', 'venue', 'batsman', 'bowler', 'striker', 'non-striker'], axis = 1, inplace = True)

In [4]:
db.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total
0,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.1,1,0,222
1,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.2,1,0,222
2,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.2,2,0,222
3,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.3,2,0,222
4,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.4,2,0,222


In [5]:
db['yyyymmdd'] = db['date'].apply(lambda x : x.split()[0])
db['year'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').year)
db['month'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').month)
db['date'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').day)
db['day_number'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').weekday())
db = db.drop(['yyyymmdd'], axis = 1)

In [6]:
db.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number
0,18,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.1,1,0,222,2008,4,4
1,18,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.2,1,0,222,2008,4,4
2,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.2,2,0,222,2008,4,4
3,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.3,2,0,222,2008,4,4
4,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.4,2,0,222,2008,4,4


In [7]:
db.bat_team.unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
       'Mumbai Indians', 'Deccan Chargers', 'Kings XI Punjab',
       'Royal Challengers Bangalore', 'Delhi Daredevils',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant'], dtype=object)

In [8]:
db.columns

Index(['date', 'bat_team', 'bowl_team', 'runs', 'wickets', 'overs',
       'runs_last_5', 'wickets_last_5', 'total', 'year', 'month',
       'day_number'],
      dtype='object')

In [9]:
for col in ['bat_team', 'bowl_team']:
    t_list = []
    for i in db[col]:
        if i == 'Delhi Daredevils':
            t_list.append('Delhi Capitals')
        elif i == 'Rising Pune Supergiants':
            t_list.append('Rising Pune Supergiant')
        else:
            t_list.append(i)
    db[col] = t_list

In [10]:
#df = db[db['year'] >= 2012]
#df = df[df['overs'] >= 5.0]

In [11]:
df = db

In [12]:
df.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number
0,18,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.1,1,0,222,2008,4,4
1,18,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.2,1,0,222,2008,4,4
2,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.2,2,0,222,2008,4,4
3,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.3,2,0,222,2008,4,4
4,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.4,2,0,222,2008,4,4


In [13]:
dummy = ['bat_team', 'bowl_team']
df_dummy = pd.get_dummies(df[dummy], prefix_sep = '_', drop_first = True)
df = df.drop(dummy, axis = 1)
df = pd.concat([df, df_dummy], axis = 1)

In [14]:
df.head()

,date,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number,bat_team_Deccan Chargers,bat_team_Delhi Capitals,bat_team_Gujarat Lions,bat_team_Kings XI Punjab,bat_team_Kochi Tuskers Kerala,bat_team_Kolkata Knight Riders,bat_team_Mumbai Indians,bat_team_Pune Warriors,bat_team_Rajasthan Royals,bat_team_Rising Pune Supergiant,bat_team_Royal Challengers Bangalore,bat_team_Sunrisers Hyderabad,bowl_team_Deccan Chargers,bowl_team_Delhi Capitals,bowl_team_Gujarat Lions,bowl_team_Kings XI Punjab,bowl_team_Kochi Tuskers Kerala,bowl_team_Kolkata Knight Riders,bowl_team_Mumbai Indians,bowl_team_Pune Warriors,bowl_team_Rajasthan Royals,bowl_team_Rising Pune Supergiant,bowl_team_Royal Challengers Bangalore,bowl_team_Sunrisers Hyderabad
0,18,1,0,0.1,1,0,222,2008,4,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,18,1,0,0.2,1,0,222,2008,4,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,18,2,0,0.2,2,0,222,2008,4,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,18,2,0,0.3,2,0,222,2008,4,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,18,2,0,0.4,2,0,222,2008,4,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [15]:
df.shape

(76014, 34)

In [16]:
df.columns

Index(['date', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5',
       'total', 'year', 'month', 'day_number', 'bat_team_Deccan Chargers',
       'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions',
       'bat_team_Kings XI Punjab', 'bat_team_Kochi Tuskers Kerala',
       'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians',
       'bat_team_Pune Warriors', 'bat_team_Rajasthan Royals',
       'bat_team_Rising Pune Supergiant',
       'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad',
       'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals',
       'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab',
       'bowl_team_Kochi Tuskers Kerala', 'bowl_team_Kolkata Knight Riders',
       'bowl_team_Mumbai Indians', 'bowl_team_Pune Warriors',
       'bowl_team_Rajasthan Royals', 'bowl_team_Rising Pune Supergiant',
       'bowl_team_Royal Challengers Bangalore',
       'bowl_team_Sunrisers Hyderabad'],
      dtype='object')

In [16]:
df = df[['runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'total', 'year', 'bat_team_Deccan Chargers', 'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions', 
         'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Pune Warriors', 
         'bat_team_Rajasthan Royals', 'bat_team_Rising Pune Supergiant', 'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad', 
         'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals', 'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders', 
         'bowl_team_Mumbai Indians', 'bowl_team_Pune Warriors', 'bowl_team_Rajasthan Royals', 'bowl_team_Rising Pune Supergiant', 'bowl_team_Royal Challengers Bangalore', 
         'bowl_team_Sunrisers Hyderabad']]

In [17]:
train = df[df['year'] <= 2016]
test = df[df['year'] > 2016]

In [18]:
X_train = train.drop(['year', 'total'], axis = 1)
X_test = test.drop(['year', 'total'], axis = 1)
y_train = train['total']
y_test = test['total']

In [19]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((68862, 27), (7152, 27), (68862,), (7152,))

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [21]:
y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)
print('Logistic Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_lr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_lr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_lr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_lr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_lr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_lr) / y_test))

Logistic Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5206602546350326
Test r2_score:  0.534061307784617
----------------------------------------------------------------------------------------------------
Train RMSE:  20.165599500438937
Test RMSE:  20.238953138665543
----------------------------------------------------------------------------------------------------
Train MAPE:  0.09717050888021991
Test MAPE:  0.09427843100236644


In [22]:
data_lr = pd.DataFrame()
data_lr['actual'] = y_test
data_lr['predicted'] = y_test_pred_lr
data_lr['lower range'] = y_test_pred_lr - (0.09427843100236644 * y_test_pred_lr)
data_lr['upper range'] = y_test_pred_lr + (0.09427843100236644 * y_test_pred_lr)
t_list = []
for i in data_lr.index:
    if data_lr['actual'][i] > data_lr['lower range'][i]:
        if data_lr['actual'][i] < data_lr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_lr['Bool'] = t_list
data_lr['Bool'].value_counts()/data_lr.shape[0]

True     0.620106
False    0.379894
Name: Bool, dtype: float64

In [23]:
la = Lasso()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
la_rscv = RandomizedSearchCV(estimator = la, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
la_rscv.fit(X_train, y_train)
la_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] alpha=0.624 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.2s
[CV] alpha=0.624 .....................................................
[CV] .

[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.2s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.2s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.2s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.2s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.2s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.2s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.2s
[CV] a

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   21.5s finished


{'alpha': 0.279}

In [24]:
y_train_pred_la = la_rscv.predict(X_train)
y_test_pred_la = la_rscv.predict(X_test)
print('Lasso Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_la))
print('Test r2_score: ', r2_score(y_test, y_test_pred_la))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_la)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_la)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_la) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_la) / y_test))

Lasso Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5108667545342995
Test r2_score:  0.5360424821076555
----------------------------------------------------------------------------------------------------
Train RMSE:  20.370561876137163
Test RMSE:  20.195879222637874
----------------------------------------------------------------------------------------------------
Train MAPE:  0.09732854669287187
Test MAPE:  0.09303737459108365


In [25]:
data_la = pd.DataFrame()
data_la['actual'] = y_test
data_la['predicted'] = y_test_pred_la
data_la['lower range'] = y_test_pred_la - (0.09303737459108365 * y_test_pred_la)
data_la['upper range'] = y_test_pred_la + (0.09303737459108365 * y_test_pred_la)
t_list = []
for i in data_la.index:
    if data_la['actual'][i] > data_la['lower range'][i]:
        if data_la['actual'][i] < data_la['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_la['Bool'] = t_list
data_la['Bool'].value_counts()/data_la.shape[0]

True     0.631711
False    0.368289
Name: Bool, dtype: float64

In [26]:
ri = Ridge()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
ri_rscv = RandomizedSearchCV(estimator = ri, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
ri_rscv.fit(X_train, y_train)
ri_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] .

[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.0s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.0s
[CV] a

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.7s finished


{'alpha': 0.942}

In [27]:
y_train_pred_ri = ri_rscv.predict(X_train)
y_test_pred_ri = ri_rscv.predict(X_test)
print('Ridge Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_ri))
print('Test r2_score: ', r2_score(y_test, y_test_pred_ri))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_ri)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_ri)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_ri) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_ri) / y_test))

Ridge Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5206602517038694
Test r2_score:  0.5340855096877972
----------------------------------------------------------------------------------------------------
Train RMSE:  20.16559956209527
Test RMSE:  20.238427503476533
----------------------------------------------------------------------------------------------------
Train MAPE:  0.09717001686842104
Test MAPE:  0.0942745696150554


In [28]:
data_ri = pd.DataFrame()
data_ri['actual'] = y_test
data_ri['predicted'] = y_test_pred_ri
data_ri['lower range'] = y_test_pred_ri - (0.0942745696150554 * y_test_pred_ri)
data_ri['upper range'] = y_test_pred_ri + (0.0942745696150554 * y_test_pred_ri)
t_list = []
for i in data_ri.index:
    if data_ri['actual'][i] > data_ri['lower range'][i]:
        if data_ri['actual'][i] < data_ri['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_ri['Bool'] = t_list
data_ri['Bool'].value_counts()/data_ri.shape[0]

True     0.619966
False    0.380034
Name: Bool, dtype: float64

In [39]:
abr = AdaBoostRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
                             'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
abr_rscv = RandomizedSearchCV(estimator = abr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
abr_rscv.fit(X_train, y_train)
abr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] n_estimators=200, learning_rate=0.544 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ n_estimators=200, learning_rate=0.544, total=   3.2s
[CV] n_estimators=200, learning_rate=0.544 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s


[CV] ............ n_estimators=200, learning_rate=0.544, total=   4.2s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   3.0s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   3.7s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   4.0s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   3.5s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   3.6s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   5.0s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] .

[CV] ............ n_estimators=250, learning_rate=0.229, total=   8.3s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   3.0s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   3.5s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   2.4s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   2.2s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   2.2s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   3.2s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 17.9min finished


{'n_estimators': 350, 'learning_rate': 0.021}

In [40]:
y_train_pred_abr = abr_rscv.predict(X_train)
y_test_pred_abr = abr_rscv.predict(X_test)
print('Ada Boost Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_abr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_abr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_abr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_abr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_abr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_abr) / y_test))

Ada Boost Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.3735506640076639
Test r2_score:  0.3471515062212125
----------------------------------------------------------------------------------------------------
Train RMSE:  23.05326241230492
Test RMSE:  23.956858066173687
----------------------------------------------------------------------------------------------------
Train MAPE:  0.11558111492391866
Test MAPE:  0.11465422266062672


In [41]:
data_abr = pd.DataFrame()
data_abr['actual'] = y_test
data_abr['predicted'] = y_test_pred_abr
data_abr['lower range'] = y_test_pred_abr - (0.11465422266062672 * y_test_pred_abr)
data_abr['upper range'] = y_test_pred_abr + (0.11465422266062672 * y_test_pred_abr)
t_list = []
for i in data_abr.index:
    if data_abr['actual'][i] > data_abr['lower range'][i]:
        if data_abr['actual'][i] < data_abr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_abr['Bool'] = t_list
data_abr['Bool'].value_counts()/data_abr.shape[0]

True     0.600951
False    0.399049
Name: Bool, dtype: float64

In [22]:
knr = KNeighborsRegressor()
possible_parameter_values = {'n_neighbors' : [int(x) for x in np.arange(start = 1, stop = 11, step = 1)],
                             'weights' : ['uniform', 'distance']}
knr_rscv = RandomizedSearchCV(estimator = knr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
knr_rscv.fit(X_train, y_train)
knr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] weights=uniform, n_neighbors=8 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... weights=uniform, n_neighbors=8, total=   0.8s
[CV] weights=uniform, n_neighbors=8 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV] ................... weights=uniform, n_neighbors=8, total=   0.8s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.7s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.8s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.8s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.8s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.8s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.8s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] .

[CV] ................... weights=uniform, n_neighbors=5, total=   0.7s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.8s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.7s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.8s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.7s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.7s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.8s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.3min finished


{'weights': 'uniform', 'n_neighbors': 8}

In [24]:
y_train_pred_knr = knr_rscv.predict(X_train)
y_test_pred_knr = knr_rscv.predict(X_test)
print('K Neighbors Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_knr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_knr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_knr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_knr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_knr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_knr) / y_test))

K Neighbors Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.7302635717837929
Test r2_score:  0.45298354313194744
----------------------------------------------------------------------------------------------------
Train RMSE:  15.127223321768287
Test RMSE:  21.92925373982048
----------------------------------------------------------------------------------------------------
Train MAPE:  0.06834524627021053
Test MAPE:  0.10216348804867138


In [25]:
data_knr = pd.DataFrame()
data_knr['actual'] = y_test
data_knr['predicted'] = y_test_pred_knr
data_knr['lower range'] = y_test_pred_knr - (0.10216348804867138 * y_test_pred_knr)
data_knr['upper range'] = y_test_pred_knr + (0.10216348804867138 * y_test_pred_knr)
t_list = []
for i in data_knr.index:
    if data_knr['actual'][i] > data_knr['lower range'][i]:
        if data_knr['actual'][i] < data_knr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_knr['Bool'] = t_list
data_knr['Bool'].value_counts()/data_knr.shape[0]

True     0.633949
False    0.366051
Name: Bool, dtype: float64

In [ ]:
xgbr = XGBRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)], 
                             'use_label_encoder' : [True, False], 
                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)], 
                             'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
xgbr_rscv = RandomizedSearchCV(estimator = xgbr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, 
                               random_state = 17, n_jobs = 1)
xgbr_rscv.fit(X_train, y_train)
xgbr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=  12.8s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.7s remaining:    0.0s


[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=  11.7s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=  12.0s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=  12.0s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=  10.6s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=  10.7s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=  10.9s
[CV] use

[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=  16.2s
[CV] use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971 
[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=  18.4s
[CV] use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971 
[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=  16.4s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=  16.7s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=  14.9s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=  15.6s
[CV

In [27]:
y_train_pred_xgbr = xgbr_rscv.predict(X_train)
y_test_pred_xgbr = xgbr_rscv.predict(X_test)
print('XGBoost Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_xgbr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_xgbr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_xgbr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_xgbr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_xgbr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_xgbr) / y_test))

XGBoost Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.9235852851017439
Test r2_score:  0.41389287526253915
----------------------------------------------------------------------------------------------------
Train RMSE:  8.051520735953781
Test RMSE:  22.699283920280294
----------------------------------------------------------------------------------------------------
Train MAPE:  0.02779792766537468
Test MAPE:  0.10590305756468572


In [28]:
data_xgbr = pd.DataFrame()
data_xgbr['actual'] = y_test
data_xgbr['predicted'] = y_test_pred_xgbr
data_xgbr['lower range'] = y_test_pred_xgbr - (0.10590305756468572 * y_test_pred_xgbr)
data_xgbr['upper range'] = y_test_pred_xgbr + (0.10590305756468572 * y_test_pred_xgbr)
t_list = []
for i in data_xgbr.index:
    if data_xgbr['actual'][i] > data_xgbr['lower range'][i]:
        if data_xgbr['actual'][i] < data_xgbr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_xgbr['Bool'] = t_list
data_xgbr['Bool'].value_counts()/data_xgbr.shape[0]

True     0.625419
False    0.374581
Name: Bool, dtype: float64

In [ ]:
'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'total', 'year', 'bat_team_Deccan Chargers', 'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions', 
         'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Pune Warriors', 
         'bat_team_Rajasthan Royals', 'bat_team_Rising Pune Supergiant', 'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad', 
         'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals', 'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders', 
         'bowl_team_Mumbai Indians', 'bowl_team_Pune Warriors', 'bowl_team_Rajasthan Royals', 'bowl_team_Rising Pune Supergiant', 'bowl_team_Royal Challengers Bangalore', 
         'bowl_team_Sunrisers Hyderabad'

In [21]:
knr_rscv = KNeighborsRegressor(weights = 'uniform', n_neighbors = 8)
knr_rscv.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=8)

In [101]:
arr = np.array(['runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'bat_team_Deccan Chargers', 'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions', 
         'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Pune Warriors', 
         'bat_team_Rajasthan Royals', 'bat_team_Rising Pune Supergiant', 'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad', 
         'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals', 'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders', 
         'bowl_team_Mumbai Indians', 'bowl_team_Pune Warriors', 'bowl_team_Rajasthan Royals', 'bowl_team_Rising Pune Supergiant', 'bowl_team_Royal Challengers Bangalore', 
         'bowl_team_Sunrisers Hyderabad'])

In [102]:
arr = arr.reshape(1, -1)

In [103]:
s = knr_rscv.predict(arr)[0]
s

167.125

In [104]:
s - (0.09076316522754599 * s)

151.95620601134638

In [105]:
s + (0.09076316522754599 * s)

182.29379398865362